In [ ]:
from attendance_registry import Assistance
from attendance_registry._types import ACC_EVT_DATA_FIELD as COLUMNS
from typing import Literal
import pandas as pd

In [ ]:
# Nombres de dispositivo
DEVICES = Literal['csl', 'sjc']

In [ ]:
# Creación de instancia
registry = Assistance[DEVICES](
    {
        'csl': 'G97954302',
        'sjc': 'G97954418'
    }
)

In [ ]:
# Obtención de datos
data = registry.get_daily_attendance(('2025-11-01', '2025-11-30'))

In [ ]:
(
    data
    .assign(
        **{
            # Obtención de fecha
            'date': lambda df: df['time'].astype(str).apply(lambda value: value.split(' ')[0]),
            # Obtención de hora
            'time': lambda df: df['time'].astype(str).apply(lambda value: value.split(' ')[1].split('-')[0]),
        }
    )
    .pipe(
        lambda df: (
            df
            # Ordenamiento de [fecha, tiempo] en orden [ascendente, ascendente]
            .sort_values([COLUMNS.DATE, COLUMNS.TIME])

            .assign(
                **{
                    # Evaluación de registros de entrada que se etiquetan como duplicados a partir de la segunda ocurrencia
                    'check_in_duplicated': lambda _df: (
                        (
                            # Evaluación duplicados a partir de la segunda ocurrencia de la combinación de los tres valores de usuario, fecha y tipo de registro
                            _df[[COLUMNS.USER_ID, COLUMNS.DATE, COLUMNS.REGISTRY_TYPE]]
                            .duplicated()
                        )
                        & (
                            # El tipo de registro debe ser de entrada
                            _df[COLUMNS.REGISTRY_TYPE] == 'checkIn'
                        )
                    )
                }
            )

            # Ordenamiento de [fecha, tiempo] en otden [ascendente, descendente]
            .sort_values([COLUMNS.DATE, COLUMNS.TIME], ascending= [True, False])
            .assign(
                **{
                    # Evaluación de registros de salida que se etiquetan como duplicados menos la última ocurrencia
                    'check_out_duplicated': lambda _df: (
                        (
                            _df[[COLUMNS.USER_ID, COLUMNS.DATE, COLUMNS.REGISTRY_TYPE]]
                            .duplicated()
                        ) & (
                            _df[COLUMNS.REGISTRY_TYPE] == 'checkOut'
                        )
                    )
                }
            )

            .assign(
                **{
                    # Se unen todos los valores de duplicado en una sola columna
                    'duplicated': lambda _df: _df['check_in_duplicated'] | _df['check_out_duplicated']
                }
            )

            # Ordenamiento de [fecha, tiempo] en orden [ascendente, ascendente]
            .sort_values([COLUMNS.DATE, COLUMNS.TIME])
        )
    )
    # Selección de columnas
    [[
        COLUMNS.USER_ID,
        COLUMNS.NAME,
        COLUMNS.TIME,
        COLUMNS.DATE,
        COLUMNS.REGISTRY_TYPE,
        COLUMNS.DEVICE,
        'duplicated',
    ]]
    # Exportación a Excel
    .to_excel('monthly_registry.xlsx')
)